# HQ Transit Corridor/Major Transit Stop Debugging


In [1]:
import geopandas as gpd
import lookback_wrappers
import numpy as np
import pandas as pd

from update_vars import (
    AM_PEAK,
    GCS_FILE_PATH,
    EXPORT_PATH,
    HQ_TRANSIT_THRESHOLD,
    MS_TRANSIT_THRESHOLD,
    PM_PEAK,
    PROJECT_CRS,
    SEGMENT_BUFFER_METERS,
    analysis_date,
    MPO_DATA_PATH
)

from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

/opt/conda/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [19]:
from calitp_data_analysis import geography_utils, get_fs, utils
from shared_utils import portfolio_utils

In [3]:
fs = get_fs()

In [13]:
df = pd.read_parquet('./share_counts.parquet').rename(columns={0:'share_count'}).reset_index(names='share_count_key')

In [17]:
df[['schedule_gtfs_dataset_key', 'route_dir_1', 'route_dir_2']] = df.share_count_key.str.split('__', n=2, expand=True)

In [18]:
df

,share_count_key,share_count,schedule_gtfs_dataset_key,route_dir_1,route_dir_2
0,0089bd1b0a2b78a8590d8749737d7146__17_0__22_1,1,0089bd1b0a2b78a8590d8749737d7146,17_0,22_1
1,0089bd1b0a2b78a8590d8749737d7146__17_0__36_0,2,0089bd1b0a2b78a8590d8749737d7146,17_0,36_0
2,0089bd1b0a2b78a8590d8749737d7146__17_0__36_1,2,0089bd1b0a2b78a8590d8749737d7146,17_0,36_1
3,0089bd1b0a2b78a8590d8749737d7146__17_0__71_0,2,0089bd1b0a2b78a8590d8749737d7146,17_0,71_0
4,0089bd1b0a2b78a8590d8749737d7146__17_0__71_1,2,0089bd1b0a2b78a8590d8749737d7146,17_0,71_1
...,...,...,...,...,...
10429,f8e4fa18131802bf978177326377241d__30R_1__10R_1,1,f8e4fa18131802bf978177326377241d,30R_1,10R_1
10430,f8e4fa18131802bf978177326377241d__30R_0__4_1,1,f8e4fa18131802bf978177326377241d,30R_0,4_1
10431,f8e4fa18131802bf978177326377241d__30R_1__4_1,1,f8e4fa18131802bf978177326377241d,30R_1,4_1
10432,f8e4fa18131802bf978177326377241d__4_1__30R_0,1,f8e4fa18131802bf978177326377241d,4_1,30R_0


In [20]:
publish_export = portfolio_utils.standardize_operator_info_for_exports(df, analysis_date).drop(
    columns=["base64_url", "source_record_id"]
)

In [23]:
marin_or_ggt = publish_export.query('name.str.contains("Marin") | name.str.contains("Golden Gate")')

In [35]:
def query_route_sets(df, route_ids: list):

    dir_0 = [route_id + '_0' for route_id in route_ids]
    dir_1 = [route_id + '_1' for route_id in route_ids]
    both_dirs = dir_0 + dir_1

    return df[df['route_dir_1'].isin(both_dirs) & df['route_dir_2'].isin(both_dirs)].sort_values('share_count', ascending=False)

In [36]:
query_route_sets(marin_or_ggt, ['130', '150'])

,share_count_key,share_count,schedule_gtfs_dataset_key,route_dir_1,route_dir_2,name,caltrans_district,analysis_name
9751,f3c9434e3dccc3548b8c3a967d734cd3__130_0__150_0,11,f3c9434e3dccc3548b8c3a967d734cd3,130_0,150_0,Bay Area 511 Golden Gate Transit Schedule,04 - Bay Area / Oakland,"Golden Gate Bridge, Highway and Transportation..."
9753,f3c9434e3dccc3548b8c3a967d734cd3__150_0__130_0,11,f3c9434e3dccc3548b8c3a967d734cd3,150_0,130_0,Bay Area 511 Golden Gate Transit Schedule,04 - Bay Area / Oakland,"Golden Gate Bridge, Highway and Transportation..."
9758,f3c9434e3dccc3548b8c3a967d734cd3__130_1__150_1,2,f3c9434e3dccc3548b8c3a967d734cd3,130_1,150_1,Bay Area 511 Golden Gate Transit Schedule,04 - Bay Area / Oakland,"Golden Gate Bridge, Highway and Transportation..."
9759,f3c9434e3dccc3548b8c3a967d734cd3__150_1__130_1,2,f3c9434e3dccc3548b8c3a967d734cd3,150_1,130_1,Bay Area 511 Golden Gate Transit Schedule,04 - Bay Area / Oakland,"Golden Gate Bridge, Highway and Transportation..."


In [37]:
query_route_sets(marin_or_ggt, ['22', '36', '71'])

,share_count_key,share_count,schedule_gtfs_dataset_key,route_dir_1,route_dir_2,name,caltrans_district,analysis_name
7,0089bd1b0a2b78a8590d8749737d7146__22_1__36_1,6,0089bd1b0a2b78a8590d8749737d7146,22_1,36_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
15,0089bd1b0a2b78a8590d8749737d7146__36_1__22_1,6,0089bd1b0a2b78a8590d8749737d7146,36_1,22_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
6,0089bd1b0a2b78a8590d8749737d7146__22_1__36_0,5,0089bd1b0a2b78a8590d8749737d7146,22_1,36_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
11,0089bd1b0a2b78a8590d8749737d7146__36_0__22_1,5,0089bd1b0a2b78a8590d8749737d7146,36_0,22_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
12,0089bd1b0a2b78a8590d8749737d7146__36_0__71_0,5,0089bd1b0a2b78a8590d8749737d7146,36_0,71_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
20,0089bd1b0a2b78a8590d8749737d7146__71_0__36_0,5,0089bd1b0a2b78a8590d8749737d7146,71_0,36_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
17,0089bd1b0a2b78a8590d8749737d7146__36_1__71_1,4,0089bd1b0a2b78a8590d8749737d7146,36_1,71_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
25,0089bd1b0a2b78a8590d8749737d7146__71_1__36_1,4,0089bd1b0a2b78a8590d8749737d7146,71_1,36_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
8,0089bd1b0a2b78a8590d8749737d7146__22_1__71_0,1,0089bd1b0a2b78a8590d8749737d7146,22_1,71_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
9,0089bd1b0a2b78a8590d8749737d7146__22_1__71_1,1,0089bd1b0a2b78a8590d8749737d7146,22_1,71_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District


In [38]:
query_route_sets(marin_or_ggt, ['35', '36', '23'])

,share_count_key,share_count,schedule_gtfs_dataset_key,route_dir_1,route_dir_2,name,caltrans_district,analysis_name
67,0089bd1b0a2b78a8590d8749737d7146__36_1__35_0,8,0089bd1b0a2b78a8590d8749737d7146,36_1,35_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
60,0089bd1b0a2b78a8590d8749737d7146__23_1__35_0,8,0089bd1b0a2b78a8590d8749737d7146,23_1,35_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
61,0089bd1b0a2b78a8590d8749737d7146__23_1__36_1,8,0089bd1b0a2b78a8590d8749737d7146,23_1,36_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
63,0089bd1b0a2b78a8590d8749737d7146__35_0__23_1,8,0089bd1b0a2b78a8590d8749737d7146,35_0,23_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
64,0089bd1b0a2b78a8590d8749737d7146__35_0__36_1,8,0089bd1b0a2b78a8590d8749737d7146,35_0,36_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
66,0089bd1b0a2b78a8590d8749737d7146__36_1__23_1,8,0089bd1b0a2b78a8590d8749737d7146,36_1,23_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
49,0089bd1b0a2b78a8590d8749737d7146__35_1__36_0,4,0089bd1b0a2b78a8590d8749737d7146,35_1,36_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
51,0089bd1b0a2b78a8590d8749737d7146__36_0__35_1,4,0089bd1b0a2b78a8590d8749737d7146,36_0,35_1,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
47,0089bd1b0a2b78a8590d8749737d7146__23_0__36_0,2,0089bd1b0a2b78a8590d8749737d7146,23_0,36_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
74,0089bd1b0a2b78a8590d8749737d7146__36_0__35_0,2,0089bd1b0a2b78a8590d8749737d7146,36_0,35_0,Bay Area 511 Marin Schedule,04 - Bay Area / Oakland,Marin County Transit District
